# pycarta Service Development

This notebook demonstrates how to create REST APIs using pycarta's service framework.

## Prerequisites

- Valid Carta authentication (see `01_authentication.ipynb`)
- Understanding of REST APIs and HTTP methods
- Basic knowledge of FastAPI (optional, but helpful)

## Setup

In [ ]:
import pycarta as pc

from fastapi import HTTPException
from pydantic import BaseModel, EmailStr
from requests import HTTPError
from typing import List, Optional
import json
import logging
import time

# Ensure you're authenticated
pc.login(environment="development", interactive=True)

# Agent is useful for making calls to carta services.
agent = pc.get_agent()

# Create a user namespace in which you can create test services
mynamespace = pc.get_agent().username

# Some commands produce a lot of logging output. Change logging settings to quiet this down a bit
def suppress_logging(level: int=logging.ERROR):
    logger = logging.getLogger()
    assert len(logger.handlers) == 1
    handler = logger.handlers[0]
    handler.setLevel(level)  # or DEBUG, INFO, CRITICAL, or FATAL
suppress_logging()

print("Setup complete - ensure authentication before running service examples")

Setup complete - ensure authentication before running service examples


## Basic Service Creation

### Your First Service

In [2]:
import asyncio
from threading import Thread


class ServiceRunner:
    def __init__(self, agent: None | pc.CartaAgent, *, timeout: int=10) -> None:
        self.agent: None | pc.CartaAgent = agent or pc.get_agent()
        self.loop = asyncio.get_event_loop()
        self.timeout: int = timeout
        self._task: None | Thread = None

    def __enter__(self):
        def run_connect_async():
            asyncio.run(pc.service.connect())
            
        self._task = Thread(target=run_connect_async, args=())
        self._task.start()
        print("Waiting for the service to start. This takes a few seconds.")
        time.sleep(5)
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if self._task is not None:
            try:
                self._task.join(timeout=10)
            except TimeoutError:
                print("Service did not stop in time.")
            self._task = None
        pc.service.cleanup()

    def _request(self, method: str, endpoint: str, **kwds):
        submit = self.agent.request(method, f"service/{endpoint}", **kwds)
        timeout = self.timeout
        while timeout > 0:
            result = self.agent.request(method, f"service/{endpoint}", params=submit.json())
            if result.json()["status"] != "RUNNING":
                return result
            else:
                timeout -= 1
                time.sleep(1)

    def get(self, endpoint: str, **kwds):
        return self._request("GET", endpoint, **kwds)
    
    def post(self, endpoint: str, **kwds):
        return self._request("POST", endpoint, **kwds)
    
    def put(self, endpoint: str, **kwds):
        return self._request("PUT", endpoint, **kwds)
    
    def patch(self, endpoint: str, **kwds):
        return self._request("PATCH", endpoint, **kwds)
    
    def delete(self, endpoint: str, **kwds):
        return self._request("DELETE", endpoint, **kwds)

In [22]:
# Create a simple GET endpoint
try:
    @pc.service(mynamespace, "hello").get()
    def world() -> str:
        """Say hello to the world."""
        return json.dumps({"message": "Hello, World!"})
except pc.exceptions.PermissionDeniedException:
    import sys
    print("You don't have permission to create this service. Please contact your plan administrator before proceeding.")
    sys.exit()

print("Simple service defined")
print("When started, this will be available at:")
print(f"{agent.url}/service/{mynamespace}/hello/")

Simple service defined
When started, this will be available at:
https://api.sandbox.carta.contextualize.us.com/service/bkappes/hello/


In [ ]:
suppress_logging()
with ServiceRunner(agent) as runner:
    result = runner.get(f"{mynamespace}/hello/world")
print(result.json())

Waiting for the service to start. This takes a few seconds.


INFO:     Started server process [87282]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:53518 (Press CTRL+C to quit)


INFO:     127.0.0.1:53521 - "GET /bkappes/hello/world HTTP/1.1" 200 OK
{'status': 'COMPLETE', 'startTime': '2025-10-01T23:44:27.266982', 'stopTime': '2025-10-01T23:44:32.339274', 'response': '"{\\"message\\": \\"Hello, World!\\"}"'}


ERROR:pycarta.services.server:Proxy error: no close frame received or sent
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [87282]
Exception in thread Thread-248 (run_connect_async):
  + Exception Group Traceback (most recent call last):
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
  |     self.run()
  |   File "/Users/bkappes/src/contextualize/pycarta/pycarta/venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
  |     _threading_Thread_run(self)
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/threading.py", line 1012, in run
  |     self._target(*self._args, **self._kwargs)
  |   File "/var/folders/2r/w5t_rv6d6zv28c1r2x_p5cc00000gn/T/ipykernel_87282/2219249050.py", line 14, in run_connect_async
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/asyncio/runners.py", line 194,

### Calculator Service Example

In [25]:
# Create a simple calculator service
@pc.service(mynamespace, "calculator").get("add/{a}/{b}")
def add_numbers(a: int, b: int) -> str:
    """Add two numbers together."""
    return json.dumps({"result": a + b})

@pc.service(mynamespace, "calculator").get("/multiply/{a}/{b}")
def multiply_numbers(a: int, b: int) -> str:
    """Multiply two numbers."""
    return json.dumps({"result": a * b})

print("Calculator service endpoints defined")
print("Endpoints:")
print("  GET /add/{a}/{b} - Add two numbers")
print("  GET /multiply/{a}/{b} - Multiply two numbers")

suppress_logging()
with ServiceRunner(agent) as runner:
    result = runner.get(f"{mynamespace}/calculator/add/5/3")
    print("Add:", json.dumps(result.json()))

    result = runner.get(f"{mynamespace}/calculator/multiply/5/3")
    print("Multiply:", json.dumps(result.json()))

Calculator service endpoints defined
Endpoints:
  GET /add/{a}/{b} - Add two numbers
  GET /multiply/{a}/{b} - Multiply two numbers
Waiting for the service to start. This takes a few seconds.


INFO:     Started server process [87282]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:53538 (Press CTRL+C to quit)


INFO:     127.0.0.1:53540 - "GET /bkappes/calculator/add/5/3 HTTP/1.1" 200 OK
Add: {"status": "COMPLETE", "startTime": "2025-10-01T23:46:46.607193", "stopTime": "2025-10-01T23:46:52.630214", "response": "\"{\\\"result\\\": 8}\""}
INFO:     127.0.0.1:53542 - "GET /bkappes/calculator/multiply/5/3 HTTP/1.1" 200 OK
Multiply: {"status": "COMPLETE", "startTime": "2025-10-01T23:46:54.725262", "stopTime": "2025-10-01T23:46:59.697577", "response": "\"{\\\"result\\\": 15}\""}


ERROR:pycarta.services.server:Proxy error: no close frame received or sent
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [87282]
Exception in thread Thread-280 (run_connect_async):
  + Exception Group Traceback (most recent call last):
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
  |     self.run()
  |   File "/Users/bkappes/src/contextualize/pycarta/pycarta/venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
  |     _threading_Thread_run(self)
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/threading.py", line 1012, in run
  |     self._target(*self._args, **self._kwargs)
  |   File "/var/folders/2r/w5t_rv6d6zv28c1r2x_p5cc00000gn/T/ipykernel_87282/2219249050.py", line 14, in run_connect_async
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/asyncio/runners.py", line 194,

## HTTP Methods

Support for all standard HTTP methods:

In [48]:
# Define data models
class UserCreate(BaseModel):
    name: str
    email: str
    age: int

class UserUpdate(BaseModel):
    name: None | str = None
    email: None | str = None
    age: None | int = None

print("Data models defined for user management service")

Data models defined for user management service


In [ ]:
service = pc.service(mynamespace, "users")  # root URL = {base URL}/services/api/users/, e.g. https://api.carta.contextualize.us.com/services/api/users/

# GET endpoint with path parameters
@service.get("/{user_id}")  # {root URL}/{user_id}
def get_user(user_id: int) -> str:
    return json.dumps({"user_id": user_id, "name": f"User {user_id}"})

# POST endpoint with JSON body
@service.post("/create")  # {root URL}/create
def create_user(user: UserCreate) -> str:
    return json.dumps({"message": f"Created user {user.name} <- {user.model_dump()}"})

# PUT endpoint for updates
@service.put("/{user_id}")  # {root URL}/{user_id}
def update_user(user_id: int, user: UserCreate) -> str:
    return json.dumps({"message": f"Updated user {user_id} <- {user.model_dump()}"})

# DELETE endpoint
@service.delete("/{user_id}")  # {root URL}/{user_id}
def delete_user(user_id: int) -> str:
    return json.dumps({"message": f"Deleted user {user_id}"})

# PATCH endpoint for partial updates
@service.patch("/{user_id}")  # {root URL}/{user_id}
def patch_user(user_id: int, updates: UserUpdate) -> str:
    return json.dumps({"message": f"Patched user {user_id}", "updates": updates.model_dump_json()})

print("User management service with all HTTP methods defined")

suppress_logging()
with ServiceRunner(agent) as runner:
    result = runner.get(mynamespace + "/users/1")
    print("GET:", result.json())

    result = runner.post(mynamespace + "/users/create", json={"name": "Alice", "email": "alice@example.com", "age": 30})
    print("POST:", result.json())

    result = runner.put(mynamespace + "/users/2", json={"name": "Bob", "email": "bob@example.com", "age": 25})
    print("PUT:", result.json())
    
    result = runner.patch(mynamespace + "/users/2", json={"age": 31})
    print("PATCH:", result.json())

    result = runner.delete(mynamespace + "/users/1")
    print("DELETE:", result.json())

User management service with all HTTP methods defined
Waiting for the service to start. This takes a few seconds.


INFO:     Started server process [87282]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:54030 (Press CTRL+C to quit)


GET: {'status': 'COMPLETE', 'startTime': '2025-10-02T00:33:32.594255', 'stopTime': '2025-10-02T00:33:36.997849', 'response': '"{\\"user_id\\": 1, \\"name\\": \\"User 1\\"}"'}
INFO:     127.0.0.1:54034 - "POST /bkappes/users/create HTTP/1.1" 200 OK
POST: {'status': 'COMPLETE', 'startTime': '2025-10-02T00:33:40.374322', 'stopTime': '2025-10-02T00:33:44.585974', 'response': '"{\\"message\\": \\"Created user Alice <- {\'name\': \'Alice\', \'email\': \'alice@example.com\', \'age\': 30}\\"}"'}
INFO:     127.0.0.1:54036 - "PUT /bkappes/users/2 HTTP/1.1" 200 OK
PUT: {'status': 'COMPLETE', 'startTime': '2025-10-02T00:33:47.494478', 'stopTime': '2025-10-02T00:33:52.154845', 'response': '"{\\"message\\": \\"Updated user 2 <- {\'name\': \'Bob\', \'email\': \'bob@example.com\', \'age\': 25}\\"}"'}
INFO:     127.0.0.1:54038 - "PATCH /bkappes/users/2 HTTP/1.1" 200 OK
PATCH: {'status': 'COMPLETE', 'startTime': '2025-10-02T00:33:54.814298', 'stopTime': '2025-10-02T00:33:59.063988', 'response': '"{\\"me

ERROR:pycarta.services.server:Proxy error: no close frame received or sent
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [87282]
Exception in thread Thread-446 (run_connect_async):
  + Exception Group Traceback (most recent call last):
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
  |     self.run()
  |   File "/Users/bkappes/src/contextualize/pycarta/pycarta/venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
  |     _threading_Thread_run(self)
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/threading.py", line 1012, in run
  |     self._target(*self._args, **self._kwargs)
  |   File "/var/folders/2r/w5t_rv6d6zv28c1r2x_p5cc00000gn/T/ipykernel_87282/340729537.py", line 14, in run_connect_async
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/asyncio/runners.py", line 194, 

## Advanced Routing

Path parameters, query parameters, and complex routing:

In [ ]:
data_service = pc.service(mynamespace, "data")  # root URL = {base URL}/services/api/data, e.g. https://api.carta.contextualize.us.com/services/api/data

# Path parameters
@data_service.get("/{category}/{item_id}")  # {root URL}/{category}/{item_id}
def get_item(category: str, item_id: int) -> str:
    return json.dumps({"category": category, "item_id": item_id})

# Query parameters
@data_service.get("/search")  # {root URL}/search
def search_items(*, q: str, limit: Optional[int] = 10, offset: int = 0) -> str:
    return json.dumps({
        "query": q,
        "limit": limit,
        "offset": offset,
        "results": []
    })

# Mixed parameters
@data_service.get("/categories/{category}/items")  # {root URL}/categories/{category}/items
def get_category_items(
    category: str,
    *,
    page: int = 1,
    per_page: int = 20,
    sort: str = "name"
) -> str:
    return json.dumps({
        "category": category,
        "page": page,
        "per_page": per_page,
        "sort": sort
    })

print("Advanced routing examples defined")
print("Examples of path parameters, query parameters, and mixed usage")

suppress_logging()
with ServiceRunner(agent) as runner:
    baseurl = f"{mynamespace}/data"
    result = runner.get(f"{baseurl}/A/1")
    print("Path:", result.json())

    result = runner.get(f"{baseurl}/search", params={"q": "category=A;item=1", "limit": 1})
    print("Query:", result.json())

    result = runner.get(f"{baseurl}/categories/B/items", params={"sort": "ascending"})
    print("Mixed:", result.json())

Advanced routing examples defined
Examples of path parameters, query parameters, and mixed usage
Waiting for the service to start. This takes a few seconds.


INFO:     Started server process [87282]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:54097 (Press CTRL+C to quit)


INFO:     127.0.0.1:54099 - "GET /bkappes/data/A/1 HTTP/1.1" 200 OK
Path: {'status': 'COMPLETE', 'startTime': '2025-10-02T00:41:46.335154', 'stopTime': '2025-10-02T00:41:50.990818', 'response': '"{\\"category\\": \\"A\\", \\"item_id\\": 1}"'}
INFO:     127.0.0.1:54101 - "GET /bkappes/data/search?limit=1&q=category%3DA%3Bitem%3D1 HTTP/1.1" 200 OK
Query: {'status': 'COMPLETE', 'startTime': '2025-10-02T00:41:53.735163', 'stopTime': '2025-10-02T00:41:58.156778', 'response': '"{\\"query\\": \\"category=A;item=1\\", \\"limit\\": 1, \\"offset\\": 0, \\"results\\": []}"'}
INFO:     127.0.0.1:54104 - "GET /bkappes/data/categories/B/items?sort=ascending HTTP/1.1" 200 OK
Mixed: {'status': 'COMPLETE', 'startTime': '2025-10-02T00:42:00.974843', 'stopTime': '2025-10-02T00:42:05.903548', 'response': '"{\\"category\\": \\"B\\", \\"page\\": 1, \\"per_page\\": 20, \\"sort\\": \\"ascending\\"}"'}


ERROR:pycarta.services.server:Proxy error: no close frame received or sent
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [87282]
Exception in thread Thread-518 (run_connect_async):
  + Exception Group Traceback (most recent call last):
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
  |     self.run()
  |   File "/Users/bkappes/src/contextualize/pycarta/pycarta/venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
  |     _threading_Thread_run(self)
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/threading.py", line 1012, in run
  |     self._target(*self._args, **self._kwargs)
  |   File "/var/folders/2r/w5t_rv6d6zv28c1r2x_p5cc00000gn/T/ipykernel_87282/340729537.py", line 14, in run_connect_async
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/asyncio/runners.py", line 194, 

## Authentication and Authorization

Control access to your services:

In [60]:
# Create service with specific authorization
secure_service = pc.service(mynamespace, "secureapi")  # root URL = {base URL}/services/sapi/admintools, e.g. https://api.carta.contextualize.us.com/services/sapi/admintools

# Authorize specific groups
# secure_service.authorize(groups=["MyOrg:Admins", "MyOrg:PowerUsers"], role="Editor")

# Authorize specific users
try:
    secure_service.authorize(user=pc.admin.get_current_user().email, role="User")
except KeyError:
    print("Multiple users matched, so authorization could not be set based on email.")
    print("By default, the creator of the service has access.")

@secure_service.get("/admin/stats")
def get_admin_stats() -> str:
    """Only accessible to authorized users."""
    return json.dumps({"active_users": 150, "total_services": 45})

print("Secure service with authorization defined")
print("Authorization methods commented out - uncomment when ready to use")
print("Available roles: Owner, Editor, User, Guest")

suppress_logging()
with ServiceRunner(agent) as runner:
    result = runner.get(mynamespace + "/secureapi/admin/stats")
    print("Admin stats:", result.json())

Multiple users matched, so authorization could not be set based on email.
Secure service with authorization defined
Authorization methods commented out - uncomment when ready to use
Available roles: Owner, Editor, User, Guest
Waiting for the service to start. This takes a few seconds.


INFO:     Started server process [87282]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:54181 (Press CTRL+C to quit)


INFO:     127.0.0.1:54183 - "GET /bkappes/secureapi/admin/stats HTTP/1.1" 200 OK
Admin stats: {'status': 'COMPLETE', 'startTime': '2025-10-02T00:51:58.444987', 'stopTime': '2025-10-02T00:52:05.518687', 'response': '"{\\"active_users\\": 150, \\"total_services\\": 45}"'}


ERROR:pycarta.services.server:Proxy error: no close frame received or sent
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [87282]
Exception in thread Thread-614 (run_connect_async):
  + Exception Group Traceback (most recent call last):
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
  |     self.run()
  |   File "/Users/bkappes/src/contextualize/pycarta/pycarta/venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
  |     _threading_Thread_run(self)
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/threading.py", line 1012, in run
  |     self._target(*self._args, **self._kwargs)
  |   File "/var/folders/2r/w5t_rv6d6zv28c1r2x_p5cc00000gn/T/ipykernel_87282/340729537.py", line 14, in run_connect_async
  |   File "/opt/homebrew/anaconda3/envs/py312/lib/python3.12/asyncio/runners.py", line 194, 

## Error Handling

Proper error responses with HTTP status codes:

In [5]:
error_service = pc.service(mynamespace, "errorhandling")

@error_service.get("/users/{user_id}")
def get_user_with_validation(user_id: int) -> str:
    if user_id < 0:
        raise HTTPException(status_code=400, detail="Invalid user ID")
    if user_id > 1000:
        raise HTTPException(status_code=404, detail="User not found")
    return json.dumps({"user_id": user_id, "name": f"User {user_id}"})

# Custom exception handling
class CustomException(Exception):
    def __init__(self, message: str):
        self.message = message

@error_service.get("/trigger-error")
def trigger_error():
    raise CustomException("This is a custom error")

print("Error handling examples defined")
print("Shows proper HTTP status codes and custom exceptions")

with ServiceRunner(agent) as runner:
    suppress_logging()

    try:
        runner.get(mynamespace + "/users/-1")
    except HTTPError as err:
        print("ID < 0 Error:", str(err))

    try:
        runner.get(mynamespace + "/users/1024")
    except HTTPError as err:
        print("ID > 1000 Error:", str(err))

    try:
        runner.get(mynamespace + "/trigger-error")
    except HTTPError as err:
        print("Trigger Error:", str(err))


Error handling examples defined
Shows proper HTTP status codes and custom exceptions
Waiting for the service to start. This takes a few seconds.


INFO:     Started server process [14971]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:62106 (Press CTRL+C to quit)


ID < 0 Error: 404 Client Error: Not Found for url: https://api.sandbox.carta.contextualize.us.com/service/bkappes/users/-1
ID > 1000 Error: 404 Client Error: Not Found for url: https://api.sandbox.carta.contextualize.us.com/service/bkappes/users/1024
Trigger Error: 404 Client Error: Not Found for url: https://api.sandbox.carta.contextualize.us.com/service/bkappes/trigger-error


## Complete Service Example

A real-world service with full CRUD operations:

In [6]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Data models
class User(BaseModel):
    id: Optional[int] = None
    name: str
    email: EmailStr
    age: int
    active: bool = True

class UserUpdatePartial(BaseModel):
    name: Optional[str] = None
    email: Optional[EmailStr] = None
    age: Optional[int] = None
    active: Optional[bool] = None

# In-memory store (use a real database in production)
users_db = {}
next_id = 1

print("Complete service example setup complete")
print("In-memory database initialized")

Complete service example setup complete
In-memory database initialized


In [7]:
# Create service with authorization
full_service = pc.service(mynamespace, "user-management")
# full_service.authorize(groups=["MyCompany:Developers"], role="Editor")

@full_service.get("/users", response_model=List[User])
def list_users(skip: int = 0, limit: int = 100) -> list[User]:
    """List all users with pagination."""
    users = list(users_db.values())[skip:skip + limit]
    logger.info(f"Listed {len(users)} users")
    return users

@full_service.get("/users/{user_id}", response_model=User)
def get_user_full(user_id: int) -> User:
    """Get a specific user by ID."""
    if user_id not in users_db:
        raise HTTPException(status_code=404, detail="User not found")
    return users_db[user_id]

@full_service.post("/users", response_model=User)
def create_user_full(user: User) -> User:
    """Create a new user."""
    global next_id
    
    # Check if email already exists
    for existing_user in users_db.values():
        if existing_user.email == user.email:
            raise HTTPException(status_code=400, detail="Email already exists")
    
    user.id = next_id
    users_db[next_id] = user
    next_id += 1
    
    logger.info(f"Created user {user.id}: {user.name}")
    return user

@full_service.put("/users/{user_id}", response_model=User)
def update_user_full(user_id: int, user_update: User) -> User:
    """Update an existing user."""
    if user_id not in users_db:
        raise HTTPException(status_code=404, detail="User not found")
    
    user_update.id = user_id
    users_db[user_id] = user_update
    
    logger.info(f"Updated user {user_id}")
    return user_update

@full_service.patch("/users/{user_id}", response_model=User)
def patch_user_full(user_id: int, user_update: UserUpdatePartial) -> User:
    """Partially update an existing user."""
    if user_id not in users_db:
        raise HTTPException(status_code=404, detail="User not found")
    
    existing_user = users_db[user_id]
    update_data = user_update.model_dump(exclude_unset=True)
    updated_user = existing_user.copy(update=update_data)
    users_db[user_id] = updated_user
    
    logger.info(f"Patched user {user_id}")
    return updated_user

@full_service.delete("/users/{user_id}")
def delete_user_full(user_id: int) -> str:
    """Delete a user."""
    if user_id not in users_db:
        raise HTTPException(status_code=404, detail="User not found")
    
    del users_db[user_id]
    logger.info(f"Deleted user {user_id}")
    return json.dumps({"message": f"User {user_id} deleted successfully"})

@full_service.get("/health")
def health_check() -> str:
    """Service health check."""
    return json.dumps({"status": "healthy", "users_count": len(users_db)})

print("Complete user management service defined")
print("Includes all CRUD operations, validation, and logging")

Complete user management service defined
Includes all CRUD operations, validation, and logging


## Testing the Service

Add some sample data and test endpoints:

In [8]:
# Add some sample data
sample_users = [
    User(name="Alice Johnson", email="alice@example.com", age=30),
    User(name="Bob Smith", email="bob@example.com", age=25),
    User(name="Charlie Brown", email="charlie@example.com", age=35),
]

for user in sample_users:
    user.id = next_id
    users_db[next_id] = user
    next_id += 1

print(f"Added {len(sample_users)} sample users to the database")
print(f"Current users in database: {len(users_db)}")
print("\nSample users:")
for user_id, user in users_db.items():
    print(f"  {user_id}: {user.name} ({user.email})")

Added 3 sample users to the database
Current users in database: 3

Sample users:
  1: Alice Johnson (alice@example.com)
  2: Bob Smith (bob@example.com)
  3: Charlie Brown (charlie@example.com)


## Service Deployment

How to start and deploy your services:

In [9]:
print("""
SERVICE DEPLOYMENT:

1. Local Development:
   To start your service locally, add this to your Python script:
   
   if __name__ == "__main__":
       pc.service.connect()
   
   This will:
   - Start a local server on an available port
   - Register with Carta platform
   - Make your service accessible via Carta URL

2. Service URLs:
   Your service will be available at:
   https://carta.contextualize.us.com/{namespace}/{service}/
   
   For example:
   https://carta.contextualize.us.com/mycompany/user-management/

3. Documentation:
   Automatic OpenAPI docs available at:
   https://carta.contextualize.us.com/{namespace}/{service}/docs

4. Production Deployment:
   For production, deploy to your server then register:
   
   from pycarta.admin.service import register_service
   register_service(
       namespace="production",
       service="my-app", 
       url="https://my-production-server.com"
   )
""")


SERVICE DEPLOYMENT:

1. Local Development:
   To start your service locally, add this to your Python script:

   if __name__ == "__main__":
       pc.service.connect()

   This will:
   - Start a local server on an available port
   - Register with Carta platform
   - Make your service accessible via Carta URL

2. Service URLs:
   Your service will be available at:
   https://carta.contextualize.us.com/{namespace}/{service}/

   For example:
   https://carta.contextualize.us.com/mycompany/user-management/

3. Documentation:
   Automatic OpenAPI docs available at:
   https://carta.contextualize.us.com/{namespace}/{service}/docs

4. Production Deployment:
   For production, deploy to your server then register:

   from pycarta.admin.service import register_service
   register_service(
       namespace="production",
       service="my-app", 
       url="https://my-production-server.com"
   )



## Best Practices

Guidelines for service development:

In [10]:
print("""
SERVICE DEVELOPMENT BEST PRACTICES:

1. Service Design:
   - Use meaningful, unique namespace names
   - Consider API versioning in namespace or endpoints
   - Document thoroughly with docstrings and type hints
   - Handle errors gracefully with meaningful messages

2. Security:
   - Apply principle of least privilege for authorization
   - Use Pydantic models for request validation
   - Be careful about data exposure in responses
   - Monitor and log access patterns

3. Performance:
   - Use async functions for I/O-bound operations
   - Implement appropriate caching strategies
   - Set reasonable limits on request body sizes
   - Monitor resource usage (memory, CPU)

4. Data Validation:
   - Use Pydantic models for request/response validation
   - Provide clear error messages for validation failures
   - Use appropriate HTTP status codes

5. Testing:
   - Test all endpoints with various inputs
   - Test error conditions and edge cases
   - Verify authorization works correctly
   - Test with realistic data volumes

6. Documentation:
   - Write clear docstrings for all endpoints
   - Use type hints for automatic documentation
   - Provide example requests/responses
   - Keep documentation up to date
""")


SERVICE DEVELOPMENT BEST PRACTICES:

1. Service Design:
   - Use meaningful, unique namespace names
   - Consider API versioning in namespace or endpoints
   - Document thoroughly with docstrings and type hints
   - Handle errors gracefully with meaningful messages

2. Security:
   - Apply principle of least privilege for authorization
   - Use Pydantic models for request validation
   - Be careful about data exposure in responses
   - Monitor and log access patterns

3. Performance:
   - Use async functions for I/O-bound operations
   - Implement appropriate caching strategies
   - Set reasonable limits on request body sizes
   - Monitor resource usage (memory, CPU)

4. Data Validation:
   - Use Pydantic models for request/response validation
   - Provide clear error messages for validation failures
   - Use appropriate HTTP status codes

5. Testing:
   - Test all endpoints with various inputs
   - Test error conditions and edge cases
   - Verify authorization works correctly
   - Te

## Next Steps

After creating services:
1. Set up MQTT messaging for real-time features (see `04_mqtt.ipynb`)
2. Integrate data management capabilities (see `05_data_management.ipynb`)
3. Connect with Seven Bridges workflows (see `06_seven_bridges.ipynb`)
4. Deploy to production environment
5. Monitor and maintain your services